In [94]:
import pickle
import multiprocessing as mp
import timeit

import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

justin = '../models/justinTest.pkl'
kip = '../models/Forclo_2_catboost_bag_052320.pkl'
model = '../models/xgb_model.pkl' # works
test = '../data/MLReady/FM_FULL_EPOCH3_MLReady.csv'
target = 'zeroBalCode'

p = mp.Pool(4)

In [95]:
model_inputs = pd.read_pickle(model)

print(f'Model inputs: {model_inputs}')

Model inputs: ['origChannel' 'origIntRate' 'origUPB' 'origLTV' 'numBorrowers'
 'origDebtIncRatio' 'borrCreditScore' 'loanPurp' 'zipCode' 'pMIperct'
 'mortInsType' 'bestCreditScore' 'worstCreditScore' 'avgCreditScore'
 'bankNumber' 'stateNumber' 'mSA' 'fmacRateMax' 'fmacRateMin'
 'fmacRateAvg' 'fmacRateVolatility' 'fredRate' 'rateDiffAbove'
 'rateDiffBelow' 'rateDiffAvg' 'rateDiffAbovePct' 'rateDiffBelowPct'
 'rateDiffAvgPct' 'origYear' 'origMonth']


# Load in test data

In [54]:
import random

n = sum(1 for line in open(test)) - 1 # number of records in file (excludes header)
s = 100 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) # the 0-indexed header will not be included in the skip list

dfTest = pd.read_csv(test, skiprows=skip)

dfTest.drop(['Unnamed: 0'], axis=1, inplace=True)
dfTest.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,bankNumber,stateNumber,mSA,zeroBalCode,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,1,5.250,288000,78,2,42,627,2,925,0.0,0,641,627,634,54,4,40140,0,4.35,4.16,4.2550,0.045673,2.65,0.900,-1.090,0.9950,0.206897,-0.262019,0.233843,2013,11
1,1,4.500,287000,72,1,43,791,1,281,0.0,0,791,791,791,54,28,16740,0,4.35,4.16,4.2550,0.045673,2.65,0.150,-0.340,0.2450,0.034483,-0.081731,0.057579,2013,11
2,1,4.500,293000,79,1,37,786,2,461,0.0,0,786,786,786,54,16,26900,0,4.35,4.16,4.2550,0.045673,2.65,0.150,-0.340,0.2450,0.034483,-0.081731,0.057579,2013,11
3,3,5.125,202000,67,2,40,672,1,117,0.0,0,687,672,679,30,35,35620,0,4.35,4.16,4.2550,0.045673,2.65,0.775,-0.965,0.8700,0.178161,-0.231971,0.204465,2013,11
4,3,4.625,178000,59,1,34,639,1,496,0.0,0,639,639,639,54,23,0,0,4.48,4.42,4.4575,0.013575,2.75,0.145,-0.205,0.1675,0.032366,-0.046380,0.037577,2013,12


In [76]:
# Get just the model inputs
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

final_columns = np.append(model_inputs, target)

dfTestData = select_columns(dfTest, final_columns)

dfTestData.sample(5)

,origIntRate,origUPB,origLTV,origDebtIncRatio,stateNumber,fredRate,zeroBalCode
42,4.625,104000,72,24,18,2.22,0
68,3.875,334000,69,34,4,1.83,0
92,3.800,329000,70,34,4,2.37,0
94,4.625,210000,70,36,4,2.81,0
93,4.875,149000,78,38,47,2.40,0


# Load the model

In [78]:
import joblib
the_model = joblib.load(model)

# Predict!

In [79]:
# Split the test data
X_test = dfTestData.loc[:, dfTestData.columns != 'zeroBalCode']
y_test = dfTestData['zeroBalCode']

In [81]:
result = the_model.score(X_test, Y_test)
print(result)

AttributeError: 'list' object has no attribute 'score'

In [83]:
the_model = pickle.loads(model)

TypeError: a bytes-like object is required, not 'str'

In [86]:
the_model = open(model, 'rb')
the_model.close()
type(the_model)

_io.BufferedReader

In [88]:
result = the_model.score(X_test, Y_test)
print(result)

AttributeError: '_io.BufferedReader' object has no attribute 'score'

In [91]:
import pickle, dill
the_model = dill.load(open(model, 'rb'))

result = the_model.score(X_test, Y_test)
print(result)

AttributeError: 'numpy.ndarray' object has no attribute 'score'

In [ ]:

origIntRate
origUPB
origLTV
origDebtIncRatio
stateNumber
fredRate

print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(Xtest)